In [ ]:
pip  install pyarabic

In [ ]:
pip install nltk

In [ ]:
pip install aiogoogletrans

In [ ]:
pip install unidecode

In [ ]:
pip install emoji

In [ ]:
pip install snowballstemmer

In [84]:
from __future__ import unicode_literals
import unicodedata
from unidecode import unidecode
import emoji
from aiogoogletrans import Translator
import asyncio
import string
import re
import pyarabic.araby as araby
from pyarabic.araby import strip_tatweel
from pyarabic import araby
from snowballstemmer import stemmer

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# Arabic stopwords
arabic_stopwords = stopwords.words('arabic')

get emails

In [86]:
def get_emails(text):
	emails = re.findall(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', text)
	return emails

get hashtags

In [87]:
def split_hashtag_to_words(tag):
    tag = tag.replace('#', '')
    tags = tag.split('_')
    if len(tags) > 1:
        return tags
    else:
        return [tag]

def extract_hashtag(text):
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list:
        word_list.extend(split_hashtag_to_words(word))
    return word_list


In [88]:
text = "هذا هو مثال #جيد لاختبار الوظيفة #استخراج_الوسم"
result = extract_hashtag(text)
print(result)


['جيد', 'استخراج', 'الوسم']


In [89]:
text = "هل جربت #التحدث بالإيموجي؟ يضيف للمحادثات #التعبير_ عن المشاعر بشكل ممتع! 😊📱"
result = extract_hashtag(text)
print(result)


['التحدث', 'التعبير', '']


get mentions

In [90]:
def extract_mentions_arabic(text):
    mentions = re.findall(r'@(\w+)', text)
    return mentions


text_arabic = "مرحبا @مستخدم1 و @مستخدم2، كيف حالكم؟"
mentions_arabic = extract_mentions_arabic(text_arabic)
print(mentions_arabic)  


['مستخدم1', 'مستخدم2']


3- get urls

In [91]:
def extract_urls(text):
	urls = re.findall(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',text)
	return urls


text = "قم بزيارة هذا الموقع: https://example.com، وأيضًا https://example.org"
urls = extract_urls(text)
print(urls)

[('https', 'example.com', ''), ('https', 'example.org', '')]


4- treat hashtags : remove the whole hashtag or just the character 

In [92]:
def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)


def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False



def remove_hashtag_char(text):
    mention_pattern = r'#+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return cleaned_text

In [93]:
text = "مرحبا  كيف حالك؟ منشن #dfhg#  لـ @مستخدم2 في هذا التغريدة."
cleaned_text = clean_hashtag(text)
print(cleaned_text)


مرحبا كيف حالك؟ منشن dfhg لـ @مستخدم2 في هذا التغريدة.


5- remove mentions 

In [94]:
def remove_mentions(text):
    mention_pattern = r'@+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return cleaned_text

text = "مرحبا @مستخدم1، كيف حالك؟ منشن لـ @مستخدم2 في هذا التغريدة."
cleaned_text = remove_mentions(text)
print(cleaned_text)  


مرحبا مستخدم1، كيف حالك؟ منشن لـ مستخدم2 في هذا التغريدة.


6- replace emojis "only arabic case is treated"

In [ ]:
with open('emojis.csv','r',encoding='utf-8') as f:
    lines = f.readlines()
    emojis_ar = {}
    for line in lines:
        line = line.strip('\n').split(';')
        emojis_ar.update({line[0].strip():line[1].strip()})



def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  
                                   u"\U0001F300-\U0001F5FF" 
                                   u"\U0001F680-\U0001F6FF" 
                                   u"\U0001F1E0-\U0001F1FF" 
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text


def emoji_native_translation(text):
    text = text.lower()
    loves = ["<3", "♥",'❤']
    smilefaces = []
    sadfaces = []
    neutralfaces = []

    eyes = ["8",":","=",";"]
    nose = ["'","`","-",r"\\"]
    for e in eyes:
        for n in nose:
            for s in ["\)", "d", "]", "}","p"]:
                smilefaces.append(e+n+s)
                smilefaces.append(e+s)
            for s in ["\(", "\[", "{"]:
                sadfaces.append(e+n+s)
                sadfaces.append(e+s)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(e+n+s)
                neutralfaces.append(e+s)
            #reversed
            for s in ["\(", "\[", "{"]:
                smilefaces.append(s+n+e)
                smilefaces.append(s+e)
            for s in ["\)", "\]", "}"]:
                sadfaces.append(s+n+e)
                sadfaces.append(s+e)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(s+n+e)
                neutralfaces.append(s+e)

    smilefaces = list(set(smilefaces))
    sadfaces = list(set(sadfaces))
    neutralfaces = list(set(neutralfaces))
    t = []
    for w in text.split():
        if w in loves:
            t.append("حب")
        elif w in smilefaces:
            t.append("مضحك")
        elif w in neutralfaces:
            t.append("عادي")
        elif w in sadfaces:
            t.append("محزن")
        else:
            t.append(w)
    newText = " ".join(t)
    return newText

   
def is_emoji(word):
    if word in emojis_ar:
        return True
    else:
        return False
    
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()


loop = asyncio.get_event_loop()
def translate_emojis(words):
    word_list = list()
    words_to_translate = list()
    for word in words :
        t = emojis_ar.get(word.get('emoji'),None)
        if t is None:
            word.update({'translation':'عادي','translated':True})
            #words_to_translate.append('normal')
        else:
            word.update({'translated':False,'translation':t})
            words_to_translate.append(t.replace(':','').replace('_',' '))
        word_list.append(word)
    return word_list

def emoji_unicode_translation(text):
    text = add_space(text)
    words = text.split()
    text_list = list()
    emojis_list = list()
    c = 0
    for word in words:
        if is_emoji(word):
            emojis_list.append({'emoji':word,'emplacement':c})
        else:
            text_list.append(word)
        c+=1
    emojis_translated = translate_emojis(emojis_list)
    for em in emojis_translated:
        text_list.insert(em.get('emplacement'),em.get('translation'))
    text = " ".join(text_list)
    return text
    
def clean_emoji(text):
    text = emoji_native_translation(text)
    text = emoji_unicode_translation(text)
    return text

In [96]:
text_with_emojis = "أحبك ❤️😊"
cleaned_text = clean_emoji(text_with_emojis)
print(cleaned_text)


text_with_emojis = "أنا حزين جدًا 😢"
cleaned_text = clean_emoji(text_with_emojis)
print(cleaned_text)


text_with_emojis = "أنا في حالة مزاجية جيدة 😄"
cleaned_text = clean_emoji(text_with_emojis)
print(cleaned_text)



أحبك قلب احمر خدود
أنا حزين جدًا يبكي
أنا في حالة مزاجية جيدة ابتسامة


remove emails

In [97]:
def remove_emails(text):
	return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)',"",text)

remove URLs

In [98]:
def remove_urls(text):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',"",text)
    

In [99]:
text = "تحقق من هذا الرابط: https://example.com، إنه رائع!"
cleaned_text = remove_urls(text)
print("Arabic text with URLs removed:", cleaned_text)

Arabic text with URLs removed: تحقق من هذا الرابط: ، إنه رائع!


7- remove punctuations 

In [ ]:
all_punctuation = string.punctuation

def remove_punctuations(text):
    return re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)

In [101]:
text = "تحقق من هذا الرابط: https://example.com، إنه رائع!"
cleaned_text = remove_punctuations(text)

print("text with punctuations removed:", cleaned_text)

text with punctuations removed: تحقق من هذا الرابط  https   example com  إنه رائع 


remove non used arabic characters

In [102]:
def remove_non_arabic(text):
    arabic_pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\d\s]+')
    cleaned_text = arabic_pattern.sub('', text)
    
    return cleaned_text


remove repeated characters

In [103]:
def remove_repeated_characters(text):
    repeated_pattern = re.compile(r'(\S)(\1)+', re.UNICODE)
    cleaned_text = repeated_pattern.sub(r'\1', text)
    
    return cleaned_text

In [104]:
text = "هذاااا هوووو نص مككككككرررررررررررررررررررررررررر"
cleaned_text = remove_repeated_characters(text)
print("Text with repeated characters removed:", cleaned_text)

Text with repeated characters removed: هذا هو نص مكر


11- for arabic remove tashkeel

In [105]:
text= araby.strip_tashkeel(text)

Strip tatweel from a text.

In [106]:
text = u"العـــــربية"
print(strip_tatweel(text))

العربية


normalizeArabic

In [107]:
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) 
    return text

tokenize

In [108]:
def remove_arabic_stopwords_tokenize(text):    
    tokens = araby.tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords.words()]
    
    return filtered_tokens

In [109]:
text = "هذا هو نص تجريبي يحتوي على بعض الكلمات العربية العادية"
cleaned_text = remove_arabic_stopwords_tokenize(text)
print("Text with Arabic stopwords removed:", cleaned_text)

Text with Arabic stopwords removed: ['نص', 'تجريبي', 'يحتوي', 'الكلمات', 'العربية', 'العادية']


stemming words

In [110]:
def stem_arabic_word(word):
    arabic_stemmer = stemmer("arabic")
    stemmed_word = arabic_stemmer.stemWord(word)
    return stemmed_word

word = "يأكلون"
stemmed_word = stem_arabic_word(word)
print(stemmed_word)


ياكل


remove multipe spaces

In [111]:
def remove_multiple_spaces(text):
	return ' '.join(text.split())

In [112]:
text = u"العربية لغة       جميلة."
remove_multiple_spaces(text)

'العربية لغة جميلة.'

the proposed algorithm to treat the text

In [113]:
text="user@gmail.com"
get_emails(text)

['user@gmail.com']

In [114]:
text = "هل جربت #التحدث بالإيموجي؟ يضيف للمحادثات #التعبير عن المشاعر بشكل ممتع! 😊📱"
result = extract_hashtag(text)
print(result)


['التحدث', 'التعبير']


In [115]:

text = "مرحبا @مستخدم1 و @مستخدم2، كيف حالكم؟"
mentions = extract_mentions_arabic(text)
print(mentions)

['مستخدم1', 'مستخدم2']


In [116]:
text_with_urls = "قم بزيارة هذا الموقع: https://example.com، وأيضًا https://example.org"
urls = extract_urls(text_with_urls)
print(urls)

[('https', 'example.com', ''), ('https', 'example.org', '')]


In [117]:
def clean_text(text):
    text = clean_hashtag(text)
    text = clean_emoji(text)
    text = remove_urls(text)
    text = remove_punctuations(text)
    text = remove_non_arabic(text)
    text = remove_repeated_characters(text)
    text = remove_multiple_spaces(text)
    tokens = remove_arabic_stopwords_tokenize(text)
    for i, word in enumerate(tokens):
        diacritic_removed_word = araby.strip_tashkeel(word)
        final_word = strip_tatweel(diacritic_removed_word)
        normalized_word = normalizeArabic(final_word)
        stemmed_word = stem_arabic_word(normalized_word)
        tokens[i] = stemmed_word
    return tokens

In [118]:
arabic_text_with_urls = "قم       بزيارة 12 هذا الموقع: https://example.com، و أيضًا https://example.org"
clean_text(arabic_text_with_urls)

['قم', 'زيار', '12', 'موقع', 'ايض']